In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
df=pd.read_json('/content/gdrive/MyDrive/archive/data2.gz', lines=True,compression='gzip',)    #data reading from drive
df.head()

In [4]:
df=df.iloc[:1000]                                        # I have taken 1000 samples for training 
data_new=df[['book_id','review_text']]

In [5]:
data=data_new.set_index('book_id')
data

,review_text
book_id,
23310161,Fun sequel to the original.
17290220,One of my favorite books to read to my 5 year ...
6954929,One of the best and most imaginative childrens...
460548,My daughter is loving this. Published in the 6...
11474551,A friend sent me this. Hilarious!
...,...
17063690,There was a mix-up in the puppy department! On...
18475599,A little clown falls off the circus train and ...
20518948,Who knew there existed a bilingual biography f...


In [12]:
import nltk 
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger') 
nltk.download('wordnet') 
  
  
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
  
from nltk.corpus import stopwords 
nltk.download('stopwords') 
stop_words = set(stopwords.words('english')) 
  
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'} 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
def preprocess_sentences(text): 
  text = text.lower() 
  temp_sent =[] 
  words = nltk.word_tokenize(text) 
  tags = nltk.pos_tag(words) 
  for i, word in enumerate(words): 
      if tags[i][1] in VERB_CODES:  
          lemmatized = lemmatizer.lemmatize(word, 'v') 
      else: 
          lemmatized = lemmatizer.lemmatize(word) 
      if lemmatized not in stop_words and lemmatized.isalpha(): 
          temp_sent.append(lemmatized) 
          
  datasent = ' '.join(temp_sent) 
  datasent = datasent.replace("n't", " not") 
  datasent = datasent.replace("'m", " am") 
  datasent = datasent.replace("'s", " is") 
  datasent = datasent.replace("'re", " are") 
  datasent = datasent.replace("'ll", " will") 
  datasent = datasent.replace("'ve", " have") 
  datasent = datasent.replace("'d", " would") 
  return datasent 
  
data["text_processed"]= data["review_text"].apply(preprocess_sentences) 


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer 
  
# Vectorizing pre-processed movie plots using TF-IDF 
tfidfvec = TfidfVectorizer() 
tfidf_bookid = tfidfvec.fit_transform((data["text_processed"])) 
  
# Finding cosine similarity between vectors 
from sklearn.metrics.pairwise import cosine_similarity 
cos_sim = cosine_similarity(tfidf_bookid, tfidf_bookid) 

In [27]:
cos_sim.shape

(1000, 1000)

In [20]:
indices = pd.Series(data.index) 
  
def recommendations(book_id, cosine_sim = cos_sim): 
    recommended_books = [] 
    index = indices[indices == book_id].index[0] 
    similarity_scores = pd.Series(cosine_sim[index]).sort_values(ascending = False) 
    top_3_books = list(similarity_scores.iloc[1:4].index) 
    for i in top_3_books: 
        recommended_books.append(list(data.index)[i]) 
    return recommended_books 

In [28]:
recommendations(17290220)            #recommendation of three books with resulted book_idswho have read the book with book_id 17290220

[20454095, 1265468, 304889]